In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import  OneHotEncoder
from sklearn.model_selection import  train_test_split
import seaborn as sns
import xgboost as xgb
%matplotlib inline

In [2]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [3]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
#extract nick names
import re
string = "Hello, world. This is a, test. For the, regex."

pattern = r',([^,\.]*)\.'

train_df['NickName']=train_df['Name'].apply(lambda string: re.findall(pattern, string)[0])
test_df['NickName']=test_df['Name'].apply(lambda string: re.findall(pattern, string)[0])

In [5]:
y=train_df['Survived']
X=train_df.drop(columns=['Sex',"PassengerId",'Survived','Name','Ticket','Cabin'])
X_=test_df.drop(columns=['Sex','Name','Ticket','Cabin'])

In [6]:
X_.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Age          332 non-null    float64
 3   SibSp        418 non-null    int64  
 4   Parch        418 non-null    int64  
 5   Fare         417 non-null    float64
 6   Embarked     418 non-null    object 
 7   NickName     418 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 26.3+ KB


In [7]:
X.Age.fillna(X.Age.mean(),inplace=True)
X.Embarked.fillna(X.Embarked.mode()[0],inplace=True)
X_.Age.fillna(X.Age.mean(),inplace=True)
X_.Fare.fillna(X.Fare.mean(),inplace=True)

In [8]:
X.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400
50%,3.000000,29.699118,0.000000,0.000000,14.454200
75%,3.000000,35.000000,1.000000,0.000000,31.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Age       891 non-null    float64
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
 5   Embarked  891 non-null    object 
 6   NickName  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [10]:
encoding_Emb = X.groupby('Embarked').size()
encoding_Emb = encoding_Emb/len(X)   #// calculates frequency
X['Embarked_enc'] = X.Embarked.apply(lambda x:encoding_Emb[x])
X_['Embarked_enc'] = X_.Embarked.apply(lambda x:encoding_Emb[x])

In [16]:
X=X.drop(columns=["Nick_enc"])
X_=X_.drop(columns=["Nick_enc"])

In [15]:
encoding_nick = X.groupby('NickName').size()
encoding_nick = encoding_nick/len(X)   #// calculates frequency
from collections import defaultdict
def fn():0
d=defaultdict(fn,encoding_nick)
X['Nick_enc'] = X.NickName.apply(lambda x:encoding_nick[x])
X_['Nick_enc'] = X_.NickName.apply(lambda x:d[x])

In [118]:
from xgboost import DMatrix
cat_l=['NickName','Embarked']
for i in cat_l:
    X[i]=X[i].astype("category")
    X_[i]=X_[i].astype("category")

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=.2)
d_train=DMatrix(X_train,y_train,enable_categorical=True)
d_test=DMatrix(X_test,y_test,enable_categorical=True)

In [87]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

def train(params_):
    params = {
    'objective': 'binary:logistic',
    'eta': 0.01,
    'max_depth': 8,
    'min_child_weight': 0,
    'eval_metric':'logloss',
    'subsample': 0.9,
    'colsample_bytree': 0.8,
    'seed': 0,
}
    params.update(params_)
    result = xgb.cv(params,d_train,num_boost_round=200,metrics=['auc',])
    # print(result['test-auc-mean'].mean(),result['test-auc-std'].mean())
    print(result.mean())
    return result

In [113]:
params = {
    'objective': 'binary:logistic',
    'eta': 0.01,
    'max_depth': 8,
    'min_child_weight': 0,
    'eval_metric':['logloss','auc'],
    'subsample': 0.9,
    'colsample_bytree': 0.8,
    'seed': 0,
    'scale_pos_weight':342/549
}
evallist = [(d_train, 'train'), (d_test, 'eval')]
clf=xgb.train(params,dtrain=d_train,num_boost_round=200,evals=evallist)

[0]	train-logloss:0.68795	train-auc:0.87576	eval-logloss:0.68781	eval-auc:0.92111
[1]	train-logloss:0.68268	train-auc:0.88434	eval-logloss:0.68246	eval-auc:0.92465
[2]	train-logloss:0.67786	train-auc:0.88421	eval-logloss:0.67747	eval-auc:0.92529
[3]	train-logloss:0.67480	train-auc:0.89339	eval-logloss:0.67485	eval-auc:0.92046
[4]	train-logloss:0.67021	train-auc:0.89231	eval-logloss:0.67025	eval-auc:0.91905
[5]	train-logloss:0.66700	train-auc:0.89869	eval-logloss:0.66775	eval-auc:0.92079
[6]	train-logloss:0.66227	train-auc:0.89864	eval-logloss:0.66298	eval-auc:0.92085
[7]	train-logloss:0.65936	train-auc:0.90017	eval-logloss:0.66054	eval-auc:0.92124
[8]	train-logloss:0.65485	train-auc:0.90038	eval-logloss:0.65567	eval-auc:0.92606
[9]	train-logloss:0.65239	train-auc:0.90325	eval-logloss:0.65336	eval-auc:0.92503
[10]	train-logloss:0.64799	train-auc:0.90196	eval-logloss:0.64867	eval-auc:0.92709
[11]	train-logloss:0.64355	train-auc:0.90154	eval-logloss:0.64355	eval-auc:0.92773
[12]	train-log

In [114]:
y.value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [112]:
from sklearn.metrics import classification_report

_=classification_report(y_test,clf.predict(d_test)>=.5)
print(_)

              precision    recall  f1-score   support

           0       0.84      0.96      0.90       105
           1       0.93      0.74      0.83        74

    accuracy                           0.87       179
   macro avg       0.89      0.85      0.86       179
weighted avg       0.88      0.87      0.87       179



In [104]:
p=clf.predict(d_test,)
p.min(),p.max()

(0.1222212, 0.889557)

In [145]:
id=X_['PassengerId']
d_=DMatrix(X_,feature_names=X_.columns[1:],enable_categorical=True,)

ValueError: ('feature_names must have the same length as data, ', 'expected 8, got 7')

In [127]:
pred=(clf.predict(d_)>=.5).astype("int")
submission=pd.DataFrame()
submission['PassengerId']=id
submission['Survived']=pred
submission.to_csv("submission.csv",index=False)